In [1]:
import numpy as np
import pandas as pd

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.compose import ColumnTransformer

In [6]:
url = 'https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day32-binning-and-binarization/train.csv'
df = pd.read_csv(url)[['Age','Fare','SibSp','Parch','Survived']]

In [7]:
df.dropna(inplace = True)

In [8]:
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [9]:
df['Family'] = df['SibSp'] + df['Parch']

In [10]:
df.head()

,Age,Fare,SibSp,Parch,Survived,Family
0,22.0,7.2500,1,0,0,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [11]:
df.drop(columns = ['SibSp','Parch'],inplace = True)

In [12]:
df.head()



,Age,Fare,Survived,Family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [13]:
x = df.drop(columns = ['Survived'])
y = df['Survived']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [15]:
x_train.head()

,Age,Fare,Family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


In [16]:
# without binarization

clf = DecisionTreeClassifier()

clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
accuracy_score(y_test,y_pred)

0.6363636363636364

In [19]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv = 10,scoring = 'accuracy'))

np.float64(0.6443270735524257)

In [20]:
# Applying Binarization

from sklearn.preprocessing import Binarizer

In [21]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy = False),['Family']),
],remainder = 'passthrough')
#copy = False, because i want changes in the existing column 'family'

In [22]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

In [23]:
pd.DataFrame(x_train_trf,columns = ['Family','Age','Fare'])

,Family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [24]:
clf = DecisionTreeClassifier()
clf.fit(x_train_trf, y_train)
y_pred2 = clf.predict(x_test_trf)

accuracy_score(y_test, y_pred2)


0.6363636363636364

In [26]:
x_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),x_trf,y,cv = 10, scoring ='accuracy'))

np.float64(0.6289906103286385)

Because cross_val_score() only evaluates the model — it doesn’t do preprocessing.
So you manually preprocess (fit_transform) before giving it to the model, ensuring consistent data format.